# Artwork sequence prediction

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
BASE_PATH = '/root/work/artwork_sequence/train_test_configuration'

MULTI_CONFIG_PATH = os.path.join(BASE_PATH,'multivariate')
UNI_CONFIG_PATH = os.path.join(BASE_PATH,'univariate')
UNI_PREDICT_MULTI_CONFIG_PATH = os.path.join(BASE_PATH,'univariate_predict_multiple')
EMB_CONFIG_PATH = os.path.join(BASE_PATH,'word_embeddings')
CODE_EMB_CONFIG_PATH = os.path.join(BASE_PATH,'code_embedding')


In [4]:
museum_sequence_path = {
    'x_train' : os.path.join(BASE_PATH, 'X_train.csv'),
    'x_test' : os.path.join(BASE_PATH, 'X_test.csv'),
    'x_tour' : os.path.join(BASE_PATH, 'style_tours.csv'),
    'x_style_tour_matrix' : os.path.join(BASE_PATH, 'style_tours_matrix.npy'),
    'embedding_style_tour_matrix' : os.path.join(BASE_PATH, 'embedding_style_tours_matrix.npy'),
    'x_train_matrix' : os.path.join(BASE_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(BASE_PATH, 'X_test_matrix.npy'),
     'embedding_train_matrix' : os.path.join(BASE_PATH, 'embedding_train_matrix.npy'),
    'embedding_test_matrix' : os.path.join(BASE_PATH, 'embedding_test_matrix.npy')
}
museum_sequence_path

{'embedding_style_tour_matrix': '/root/work/artwork_sequence/train_test_configuration/embedding_style_tours_matrix.npy',
 'embedding_test_matrix': '/root/work/artwork_sequence/train_test_configuration/embedding_test_matrix.npy',
 'embedding_train_matrix': '/root/work/artwork_sequence/train_test_configuration/embedding_train_matrix.npy',
 'x_style_tour_matrix': '/root/work/artwork_sequence/train_test_configuration/style_tours_matrix.npy',
 'x_test': '/root/work/artwork_sequence/train_test_configuration/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/train_test_configuration/X_test_matrix.npy',
 'x_tour': '/root/work/artwork_sequence/train_test_configuration/style_tours.csv',
 'x_train': '/root/work/artwork_sequence/train_test_configuration/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/train_test_configuration/X_train_matrix.npy'}

## Load data

In [5]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)

df_style_tours = pd.read_csv(museum_sequence_path['x_tour'], index_col=0)
x_style_tour_matrix = np.load(museum_sequence_path['x_style_tour_matrix'])

x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])

embedding_train_matrix = np.load(museum_sequence_path['embedding_train_matrix'])
embedding_test_matrix = np.load(museum_sequence_path['embedding_test_matrix'])
embedding_style_tour_matrix = np.load(museum_sequence_path['embedding_style_tour_matrix'])

**Combine code data with embedding data**

In [10]:
x_embedding_train_matrix = np.hstack((x_train_matrix, embedding_train_matrix))
x_embedding_test_matrix = np.hstack((x_test_matrix, embedding_test_matrix))
x_embedding_style_tour_matrix = np.hstack((x_style_tour_matrix, embedding_style_tour_matrix))

## Reset Tensorflow session

In [7]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [8]:
time = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
time.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [9]:
def get_tours_conf():
    
    split_time = x_train_matrix.shape[0]
    #code matrix
    X = np.concatenate((x_train_matrix, x_test_matrix))
    #embedding matrix
    X_embeddings = np.concatenate((embedding_train_matrix, embedding_test_matrix))
    #code and embedding matrix
    X_code_embeddings = np.concatenate((x_embedding_train_matrix, x_embedding_test_matrix))
    
    return (split_time, X, X_embeddings, X_code_embeddings)

In [11]:
def get_style_tour_conf():
    
    split_time = 20000
    #code matrix
    X = x_style_tour_matrix
    #embedding matrix
    X_embeddings = embedding_style_tour_matrix
    #code and embedding matrix
    X_code_embeddings = x_embedding_style_tour_matrix
    
    return (split_time, X, X_embeddings, X_code_embeddings)

In [12]:
#split_time, X, X_embeddings, X_code_embeddings = get_tours_conf()

split_time, X, X_embeddings, X_code_embeddings = get_style_tour_conf()

#Number of feature to take into account
n_influence_features=10

batch_size = 128
shuffle_buffer_size = 300

In [13]:
X.shape

(27000, 300)

In [14]:
X_embeddings.shape[1]

100

## Create models

**Define model**

In [15]:
from Sequence_prediction_factory import Sequence_prediction_multivariate, Sequence_prediction_univariate, Sequence_prediction_embeddings

In [16]:
model_univariate = Sequence_prediction_univariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size,
    CONFIG_PATH=UNI_PREDICT_MULTI_CONFIG_PATH)

In [ ]:
model_multivariate = Sequence_prediction_multivariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size,  
    n_influence_features=n_influence_features,
    CONFIG_PATH=MULTI_CONFIG_PATH)

In [ ]:
model_embeddings = Sequence_prediction_embeddings(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size,  
    X_embeddings=X_embeddings,
    CONFIG_PATH=EMB_CONFIG_PATH)

In [ ]:
model_code_embedding = Sequence_prediction_univariate(
    X=X_code_embeddings, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size,
    CONFIG_PATH=CODE_EMB_CONFIG_PATH)

In [17]:
model_prediction = model_univariate

In [ ]:
from IPython.display import clear_output
import time

default_index = 0
#length of the history
for window_size in range(3,4):
    
    #Define model with the window size
    model_prediction.set_window_size(window_size)
    tf.keras.backend.clear_session()
    model = model_prediction.get_model()
    model.define_model(conv_filter=20, lstm_filter=80, dense_filter=20, prediction_length=12)
    model.set_index(default_index)
    model.save_weights(model_prediction.get_untrained_weights_path())

    start_time = time.time()
    
    for i in range(model_prediction.get_x_features()):
        clear_output(wait=True)
        print("---------- Feature %s -------------" % (i))
        
        #Load untrained weights for next training
        model.set_index(default_index)
        model.load_weights(model_prediction.get_untrained_weights_path())
        
        #Train model with the feature i
        model.set_index(i)
        model.train_model(epochs=10, lr=1e-6)

        #Save weights
        model.save_weights(model_prediction.get_trained_weights_path())
        

    print("--- %s seconds ---" % (time.time() - start_time))

---------- Feature 210 -------------
Epoch 1/10
157/157 [==============================] - 88s 563ms/step - loss: 0.2536 - mae: 0.5447 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/10
157/157 [==============================] - 5s 29ms/step - loss: 0.0835 - mae: 0.3203 - val_loss: 0.0836 - val_mae: 0.3184
Epoch 3/10
157/157 [==============================] - 5s 30ms/step - loss: 0.0779 - mae: 0.3111 - val_loss: 0.0806 - val_mae: 0.3139
Epoch 4/10
157/157 [==============================] - 4s 29ms/step - loss: 0.0763 - mae: 0.3085 - val_loss: 0.0792 - val_mae: 0.3118
Epoch 5/10
 10/157 [>.............................] - ETA: 3s - loss: 0.0769 - mae: 0.3105